In [49]:
import numpy as np
import pygame

In [50]:
N = 256
iterations = 10

def IX(x_coord, y_coords):
        return x_coord + y_coords*N

class Fluid():
    def __init__(self, dt, diffusion, viscosity):
        global N
        self.size = N
        self.dt = dt
        self.diff = diffusion
        self.visc = viscosity

        self.s = np.zeros((self.size*self.size), dtype=np.float32)
        self.density = np.zeros((self.size*self.size), dtype=np.float32)

        self.Vx = np.zeros((self.size*self.size), dtype=np.float32)
        self.Vy = np.zeros((self.size*self.size), dtype=np.float32)
    #     Vz = []

        self.Vx0 = np.zeros((self.size*self.size), dtype=np.float32)
        self.Vy0 = np.zeros((self.size*self.size), dtype=np.float32)
    #     Vz0 = []
    
    def addDensity(self, x_coord, y_coord, amount):
        self.density[self.IX(x, y)] += amount

    def FluidCubeAddVelocity(self, x_coord, y_coord, z_coord, addVelX, addVelY, amountZ):
        index = self.IX(x, y)

        self.Vx[index] += addVelX
        self.Vy[index] += addVelY
    
    def step(self):
        N        = self.size;
        visc     = self.visc;
        diff     = self.diff;
        dt       = self.dt;
        Vx      = self.Vx;
        Vy      = self.Vy;
        Vx0     = self.Vx0;
        Vy0     = self.Vy0;
        s       = self.s;
        density = self.density;

        diffuse(1, Vx0, Vx, visc, dt)
        diffuse(2, Vy0, Vy, visc, dt)

        project(Vx0, Vy0, Vx, Vy)

        advect(1, Vx, Vx0, Vx0, Vy0, dt)
        advect(2, Vy, Vy0, Vx0, Vy0, dt)

        project(Vx, Vy, Vx0, Vy0)

        diffuse(0, s, density, diff, dt)
        advect(0, density, s, Vx, Vy, dt)
    
    
    

In [55]:
def diffuse (b, x_array, x0_array, diff, dt):
    a = dt * diff * (N - 2) * (N - 2)
    lin_solve(b, x_array, x0_array, a, 1 + 6 * a)


def lin_solve(b, x_array, x0_array, a, c):
    cRecip = 1.0 / c
    for k in range(iterations):
        for j in range(1, N-1):
            for i in range(1, N-1):
                x_array[IX(i, j)] = (x0_array[IX(i, j)]
                                + a*(    x_array[IX(i+1, j  )]
                                +x_array[IX(i-1, j  )]
                                +x_array[IX(i  , j+1)]
                                +x_array[IX(i  , j-1)]
                       )) * cRecip
    
                
                    
        set_bnd(b, x_array)

def project(velocX_array, velocY_array, p_array, div_array):
    
    for j in range(1, N-1):
        for i in range(1, N-1):
            div_array[IX(i, j)] = -0.5*(
                     velocX_array[IX(i+1, j  )]
                    -velocX_array[IX(i-1, j  )]
                    +velocY_array[IX(i  , j+1)]
                    -velocY_array[IX(i  , j-1)]
                )/N
            p_array[IX(i, j)] = 0
            
            
    set_bnd(0, div_array); 
    set_bnd(0, p_array);
    lin_solve(0, p_array, div_array, 1, 6)
    

    for j in range(1, N-1):
        for i in range(1, N-1):
            velocX_array[IX(i, j)] -= 0.5 * (  p_array[IX(i+1, j)]
                                         -p_array[IX(i-1, j)]) * N
            velocY_array[IX(i, j)] -= 0.5 * (  p_array[IX(i, j+1)]
                                         -p_array[IX(i, j-1)]) * N
            
    set_bnd(1, velocX_array)
    set_bnd(2, velocY_array)

def advect(b, d_array, d0_array,  velocX_array, velocY_array, dt):
    i0, i1, j0, j1= [float()]*4
    
    dtx = dt * (N - 2)
    dty = dt * (N - 2)
    
    s0, s1, t0, t1= [float()]*4
    tmp1, tmp2,  x, y = [float()]*4
    
    Nfloat = float(N)
    ifloat, jfloat= [float()]*2
    i, j= [int()]*2
    
    
    for j in range(1, N-1):
        for i in range(1, N-1):
            tmp1 = dtx * velocX_array[IX(i, j)]
            tmp2 = dty * velocY_array[IX(i, j)]
            x    = float(i) - tmp1 
            y    = float(j) - tmp2
            
            if (x < 0.5): x = 0.5 
            if(x > Nfloat + 0.5): x = Nfloat + 0.5
            i0 = np.floor(x) 
            i1 = i0 + 1.0
            if(y < 0.5): y = 0.5 
            if(y > Nfloat + 0.5): y = Nfloat + 0.5 
            j0 = np.floor(y)
            j1 = j0 + 1.0

            s1 = x - i0; 
            s0 = 1.0 - s1; 
            t1 = y - j0; 
            t0 = 1.0 - t1;

            i0i = int(i0)
            i1i = int(i1)
            j0i = int(j0)
            j1i = int(j1)
            
            d_array[IX(i, j)] = s0 * (( t0 *  d0_array[IX(i0i, j0i)]) +( t1 * d0_array[IX(i0i, j1i)])) + s1 * (( t0 * d0_array[IX(i1i, j0i)]) +( t1 * d0_array[IX(i1i, j1i)]))

    set_bnd(b, d_array)
    
def set_bnd(b, x_array):
    for i in range(1, N-1):
        x_array[IX(i,  0 )] = -x_array[IX(i, 1  )] if b == 2 else x_array[IX(i, 1  )]
        x_array[IX(i, N-1)] = -x_array[IX(i, N-2)] if b == 2 else x_array[IX(i, N-2)]
            
    for j in range(1, N-1):
        x_array[IX(0  , j)] = -x_array[IX(1  , j)] if b == 1 else x_array[IX(1  , j)]
        x_array[IX(N-1, j)] = -x_array[IX(N-2, j)] if b == 1 else x_array[IX(N-2, j)]

            
    x_array[IX(0, 0)]     = 0.5 * (x_array[IX(1, 0)] + x_array[IX(0, 1)])
    x_array[IX(0, N-1)]   = 0.5 * (x_array[IX(1, N-1)] + x_array[IX(0, N-2)])
    x_array[IX(N-1, 0)]   = 0.5 * (x_array[IX(N-2, 0)] + x_array[IX(N-1, 1)])
    x_array[IX(N-1, N-1)] = 0.5 * (x_array[IX(N-2, N-1)] + x_array[IX(N-1, N-2)])

In [56]:
pygame.init()
screen = pygame.display.set_mode((400, 400))
done = False

fluid = Fluid(0.1, 0, 0)

while not done:
    fluid.step()
    
    for event in pygame.event.get():
            if event.type == pygame.QUIT:
                    done = True

    pygame.display.flip()

KeyboardInterrupt: 